#Project Title: AI-Powered News Summarizer (TL;DR Agent)
Category: Productivity Agents

Name: Harshita Mudliar
SRN: PES2UG23CS918


In [1]:
# Install necessary libraries (if not already installed)
!pip install -q transformers torch

import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd

# Check if GPU is available for faster inference
device = 0 if torch.cuda.is_available() else -1
print(f"Using device: {'GPU' if device == 0 else 'CPU'}")

Using device: CPU


In [2]:
model_name = "sshleifer/distilbart-cnn-12-6"

# Load the tokenizer and model explicitly to demonstrate understanding of the pipeline
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Initialize the summarization pipeline
summarizer = pipeline(
    "summarization",
    model=model,
    tokenizer=tokenizer,
    device=device
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Device set to use cpu



The following function includes logic to handle text truncation. Since Transformer models have a maximum "context window" we ensure the input text is managed correctly.

In [5]:
def generate_tldr(text, max_len=130, min_len=30):

    # Generate summary
    summary = summarizer(
        text,
        max_length=max_len,
        min_length=min_len,
        do_sample=False, # Deterministic output for consistency
        truncation=True
    )

    return summary[0]['summary_text']

news_article1 = """
NASA’s James Webb Space Telescope has captured a stunning new image of a distant galaxy,
revealing intricate details never seen before. The galaxy, located millions of light-years
away, shows high levels of star formation and complex gas structures. Scientists claim
this discovery could reshape our understanding of how early galaxies evolved in the
infant universe. The mission, which cost over $10 billion, continues to provide
unprecedented data that challenges existing astronomical models. NASA officials stated
that this is just the beginning of a new era in deep-space observation.
"""

news_article2 = """
The Bulletin of the Atomic Scientists announced on Tuesday that the Doomsday Clock has been set to 85 seconds to midnight, the closest the symbolic clock has ever been to
 global catastrophe. Scientists cited an 'unprecedented level of risk' driven by the ongoing Russia-Ukraine war, escalating tensions between major nuclear powers, and the
 record-breaking global heatwaves of 2025.The organization specifically highlighted the role of emerging technologies, noting that the 'increasing use of artificial
 intelligence without adequate international controls' is now a primary existential threat alongside nuclear weapons and climate change. Alexandra Bell, the group’s CEO,
 stated that while the situation is dire, it is not hopeless. She urged world leaders to prioritize international cooperation over nationalistic competition to reduce the
 risks of nuclear miscalculation and environmental collapse. The clock, which began in 1947 at seven minutes to midnight, serves as a universal warning for humanity to
 address man-made threats to our survival.
"""

# Execution
result1 = generate_tldr(news_article1)
print("--- ORIGINAL TEXT ---")
print(news_article1)
print("\n--- GENERATED TL;DR ---")
print(result1)

result2 = generate_tldr(news_article2)
print("--- ORIGINAL TEXT ---")
print(news_article2)
print("\n--- GENERATED TL;DR ---")
print(result2)

Your max_length is set to 130, but your input_length is only 128. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=64)


--- ORIGINAL TEXT ---

NASA’s James Webb Space Telescope has captured a stunning new image of a distant galaxy, 
revealing intricate details never seen before. The galaxy, located millions of light-years 
away, shows high levels of star formation and complex gas structures. Scientists claim 
this discovery could reshape our understanding of how early galaxies evolved in the 
infant universe. The mission, which cost over $10 billion, continues to provide 
unprecedented data that challenges existing astronomical models. NASA officials stated 
that this is just the beginning of a new era in deep-space observation.


--- GENERATED TL;DR ---
 Scientists claim this discovery could reshape our understanding of how early galaxies evolved in the infant universe . The James Webb Space Telescope has captured a stunning new image of a distant galaxy . It shows high levels of star formation and complex gas structures .
--- ORIGINAL TEXT ---

The Bulletin of the Atomic Scientists announced on Tuesda

#Observations
The model performed particularly well on the doomsday article, correctly extracting the three main points, the war, nuclear threats and climate change. It also performed decently in the James Webb article, keeping the facts intact.
